In [1]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_EURUSD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch the latest candle
def fetch_latest_candle(symbol, timeframe):
    latest_candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    return latest_candles[0] if len(latest_candles) > 0 else None

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        # Filter features
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    # Handle missing features by filling with zeros
    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0
    
    combined_df = combined_df[selected_features]
    combined_df = combined_df.dropna()
    
    return combined_df

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 15,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to apply the custom threshold
def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

# Predict and trade for Buy or Sell
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    # Shift the data
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)
    
    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5  # Use provided threshold or default to 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    # Save predictions to CSV
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    if len(y_pred) > 1 and y_pred[-1] == 1:  # If the second-last prediction is a signal
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

def save_predictions_to_csv(df_pred, symbol, trade_type):
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    df_pred.to_csv(file_path, index=True)

# Function to process each pair with time-based trade execution
def process_pair(config, utc_from, utc_to):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    # Initialize variables for model/scaler availability
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
    
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")

    # Load feature names for buy and sell
    buy_features, sell_features = [], []
    try:
        if "buy_features_path" in config:
            with open(config["buy_features_path"], 'r') as f:
                buy_features = json.load(f)
        if "sell_features_path" in config:
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
    except Exception as e:
        logging.error(f"Error loading features for {config['symbol']}: {e}")

    # Get trade time from config
    trade_time = config.get("trade_time", {"hour": 10, "minute": 0})  # Default to 10:00 AM UTC if not provided

    try:
        while True:
            now = datetime.now(pytz.utc)
            if now.hour == trade_time["hour"] and now.minute == trade_time["minute"]:
                logging.info(f"Executing trade for {config['symbol']} at {now}")
                
                # Predict and trade for Buy if model is available
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )
                
                # Predict and trade for Sell if model is available
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )

                # Wait until the next minute to avoid multiple trades in the same minute
                time.sleep(60)
            else:
                # Sleep for 30 seconds and check again
                time.sleep(30)

    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}

    init_mt5_connection(config['login'], config['password'], config['server'])
    
    # Configuration for EURUSD with trade time set to 10:00 AM UTC
    eurusd_config = {
        "symbol": "EURUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
        "trade_time": {"hour": 10, "minute": 47}  
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json',
        "trade_time": {"hour": 10, "minute": 47} 
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
        "trade_time": {"hour": 10, "minute": 47} 
    }

    configs = [eurusd_config, gbpusd_config, usdcad_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Connected to MetaTrader 5


Exception in thread Thread-4 (process_pair):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\forex_env\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\anaconda3\envs\forex_env\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_13536\3013843928.py", line 287, in process_pair
TypeError: predict_and_trade() missing 1 required positional argument: 'trade_decisions'
Exception in thread Thread-6 (process_pair):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\forex_env\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)


MetaTrader 5 connection closed for EURUSD.
MetaTrader 5 connection closed for USDCAD.
MetaTrader 5 connection closed for GBPUSD.


In [1]:
import MetaTrader5 as mt5
import threading
from datetime import datetime
import pytz
import logging
import json
import sys
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
logging.basicConfig(filename='trading_GBPEURUSD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Add rolling features
def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
# Combine buy/sell data without handling missing features
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    df = df.dropna().reset_index(drop=True)
    
    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')

    # We assume all features are present, no need for missing feature handling
    combined_df = combined_df[selected_features]
    
    combined_df = combined_df.dropna()
    
    return combined_df


# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None or not symbol_info.visible:
        logging.error(f"Failed to get symbol info or symbol is not visible for {symbol}.")
        mt5.symbol_select(symbol, True)
        return
    
    if symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        return
    
    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 20,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")

# Function to apply the custom threshold
def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

# Predict and trade for Buy or Sell
def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features, trade_type, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    
    signals = ['WILLR_15', 'WILLR_42']

    df_processed = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    df_processed = df_processed[features]
    scaled_data = scaler.transform(df_processed)
    
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    if len(y_pred) > 1 and y_pred[-1] == 1:
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)

def save_predictions_to_csv(df_pred, symbol, trade_type):
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    df_pred.to_csv(file_path, index=True)

# Process each pair at the specified time
def process_pair(config, utc_from, utc_to):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
    
    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")

    buy_features, sell_features = [], []
    try:
        if "buy_features_path" in config:
            with open(config["buy_features_path"], 'r') as f:
                buy_features = json.load(f)
        if "sell_features_path" in config:
            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
    except Exception as e:
        logging.error(f"Error loading features for {config['symbol']}: {e}")

    trade_time = config.get("trade_time", {"hour": 10, "minute": 0})

    try:
        while True:
            now = datetime.now(pytz.utc)
            if now.hour == trade_time["hour"] and now.minute == trade_time["minute"]:
                logging.info(f"Executing trade for {config['symbol']} at {now}")
                
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )
                
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )

                time.sleep(60)
            else:
                time.sleep(30)

    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Function to initialize connection to specific MetaTrader 5 terminal
def init_mt5_connection(login, password, server, terminal_path):
    if not mt5.initialize(login=login, password=password, server=server, path=terminal_path):
        print(f"Failed to initialize MetaTrader 5 terminal {terminal_path}, error code = {mt5.last_error()}")
        return False
    print(f"Connected to MetaTrader 5 terminal {terminal_path}")
    return True

# Function to process trading for each pair on different terminal
def process_pair_on_different_account(config, utc_from, utc_to):
    terminal_path = config["terminal_path"]  # Get the terminal path for the pair
    if not init_mt5_connection(config['login'], config['password'], config['server'], terminal_path):
        return

    # Your logic to fetch and process data, predict, and trade
    # This can be your original function for trading pairs
    process_pair(config, utc_from, utc_to)
    
    mt5.shutdown()  # Close terminal connection when done
    print(f"MetaTrader 5 terminal closed for {config['symbol']}")

# Main function to configure and launch threads for each currency pair on separate terminals
def main():
    # Configurations for each currency pair on different accounts
    
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU2\terminal64.exe",  # Path to terminal for EURUSD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
        "trade_time": {"hour": 0, "minute": 4}
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU3\terminal64.exe",  # Path to terminal for GBPUSD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json',
        "trade_time": {"hour": 0, "minute": 4}
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU4\terminal64.exe",  # Path to terminal for USDCAD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
        "trade_time": {"hour": 0, "minute": 4}
    }


    # Prepare the time range for historical data fetching
    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    # Run each pair on separate threads with different accounts
    configs = [eurusd_config, gbpusd_config, usdcad_config]
    threads = []
    
    for config in configs:
        thread = threading.Thread(target=process_pair_on_different_account, args=(config, utc_from, utc_to))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


In [ ]:
    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU2\terminal64.exe",  # Path to terminal for EURUSD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0105,
        "buy_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'EURUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112buy/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112buy/feature_names.json',
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU3\terminal64.exe",  # Path to terminal for GBPUSD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sell/feature_names.json',
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader 5 IC Markets EU4\terminal64.exe",  # Path to terminal for USDCAD
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.0088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
    }
